In [10]:
!pip install pyreadr

import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Dropout, Conv1D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading From CSV

In [12]:
Sampled_train = pd.read_csv("/content/drive/MyDrive/research/dataset/train.csv")
Sampled_test = pd.read_csv("/content/drive/MyDrive/research/dataset/test.csv")
Sampled_cv = pd.read_csv('/content/drive/MyDrive/research/dataset/cv.csv')

In [13]:
Sampled_train

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,xmeas_11,xmeas_12,xmeas_13,xmeas_14,xmeas_15,xmeas_16,xmeas_17,xmeas_18,xmeas_19,xmeas_20,xmeas_21,xmeas_22,xmeas_23,xmeas_24,xmeas_25,xmeas_26,xmeas_27,xmeas_28,xmeas_29,xmeas_30,xmeas_31,xmeas_32,xmeas_33,xmeas_34,xmeas_35,xmeas_36,xmeas_37,xmeas_38,xmeas_39,xmeas_40,xmeas_41,xmv_1,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,120.41,0.33818,80.044,51.435,2632.9,25.029,50.528,3101.1,22.819,65.732,229.61,341.22,94.640,77.047,32.188,8.8933,26.383,6.8820,18.776,1.6567,32.958,13.823,23.978,1.2565,18.579,2.2633,4.8436,2.2986,0.017866,0.83570,0.098577,53.724,43.828,62.881,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,75.000,120.41,0.33620,80.078,50.154,2633.8,24.419,48.772,3102.0,23.333,65.716,230.54,341.30,94.595,77.434,32.188,8.8933,26.383,6.8820,18.776,1.6567,32.958,13.823,23.978,1.2565,18.579,2.2633,4.8436,2.2986,0.017866,0.83570,0.098577,53.724,43.828,63.132,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,74.771,120.42,0.33563,80.220,50.302,2635.5,25.244,50.071,3103.5,21.924,65.732,230.08,341.38,94.605,77.466,31.767,8.7694,26.095,6.8259,18.961,1.6292,32.985,13.742,23.897,1.3001,18.765,2.2602,4.8543,2.3900,0.017866,0.83570,0.098577,53.724,43.828,63.117,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,75.224,120.39,0.33553,80.305,49.990,2635.6,23.268,50.435,3102.8,22.948,65.781,227.91,341.71,94.473,77.443,31.767,8.7694,26.095,6.8259,18.961,1.6292,32.985,13.742,23.897,1.3001,18.765,2.2602,4.8543,2.3900,0.017866,0.83570,0.098577,53.724,43.828,63.100,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,75.388,120.39,0.32632,80.064,51.310,2632.4,26.099,50.480,3103.5,22.808,65.788,231.37,341.11,94.678,76.947,32.322,8.5821,26.769,6.8688,18.782,1.6396,33.071,13.834,24.228,1.0938,18.666,2.2193,4.8304,2.2416,0.017866,0.83570,0.098577,53.724,43.828,63.313,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270395,20.0,24.0,496,0.28437,3645.3,4476.8,9.3158,27.041,42.017,2691.9,74.217,120.41,0.33693,79.951,50.860,2610.8,24.668,48.663,3093.0,22.379,65.982,235.45,336.23,94.645,76.700,32.044,8.7699,26.246,6.8068,19.346,1.6191,32.682,13.709,23.334,1.2064,18.913,2.2850,4.7707,2.2680,0.018453,0.85368,0.088404,53.842,44.153,63.581,53.831,27.820,58.953,19.810,40.356,40.631,43.441,49.613,42.208,19.711
270396,20.0,24.0,497,0.27925,3687.0,4471.1,9.2843,26.594,41.935,2692.1,74.513,120.39,0.32284,79.890,50.685,2610.6,27.534,49.249,3090.2,23.482,65.926,238.03,336.88,94.655,76.948,32.151,8.9606,26.211,6.8488,18.977,1.7243,32.865,13.787,23.628,1.0893,19.075,2.2680,4.8855,2.2031,0.018453,0.85368,0.088404,53.842,44.153,63.320,52.904,27.161,61.933,19.796,38.746,40.117,44.795,49.616,40.490,17.089
270397,20.0,24.0,498,0.27898,3671.8,4453.8,9.2279,26.427,42.475,2694.8,74.098,120.42,0.32033,80.184,49.269,2612.9,25.033,48.594,3092.7,23.109,65.937,238.28,337.30,94.665,77.357,32.151,8.9606,26.211,6.8488,18.977,1.7243,32.865,13.787,23.628,1.0893,19.075,2.2680,4.8855,2.2031,0.018453,0.85368,0.088404,53.842,44.153,63.581,53.112,27.188,62.026,19.764,39.052,35.948,43.279,49.824,41.805,17.934
270398,20.0,24.0,499,0.26937,3691.9,4495.6,9.3308,26.934,42.480,2694.8,75.163,120.39,0.33166,80.240,50.435,2612.6,25.598,48.453,3093.8,22.824,66.016,241.27,336.74,94.604,77.187,32.118,8.8653,26.227,6.8508,19.090,1.7002,32.6

# Preprocessing

In [14]:
# Drop some mysterious fault type
Sampled_train.drop(Sampled_train[(Sampled_train.faultNumber == 3) | (Sampled_train.faultNumber == 9) | (Sampled_train.faultNumber == 15)].index, inplace = True)
Sampled_test.drop(Sampled_test[(Sampled_test.faultNumber == 3) | (Sampled_test.faultNumber == 9) | (Sampled_test.faultNumber == 15)].index, inplace = True)
Sampled_cv.drop(Sampled_cv[(Sampled_cv.faultNumber == 3) | (Sampled_cv.faultNumber == 9) | (Sampled_cv.faultNumber == 15)].index, inplace = True)

In [15]:
# make the Y value usable in LSTM
y_train = to_categorical(Sampled_train['faultNumber'],num_classes=21)
y_test = to_categorical(Sampled_test['faultNumber'],num_classes=21)
y_cv = to_categorical(Sampled_cv['faultNumber'],num_classes=21)

In [16]:
# Test if the dropping is successful
Sampled_cv[Sampled_cv['faultNumber'] == 1]
Sampled_train

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,xmeas_11,xmeas_12,xmeas_13,xmeas_14,xmeas_15,xmeas_16,xmeas_17,xmeas_18,xmeas_19,xmeas_20,xmeas_21,xmeas_22,xmeas_23,xmeas_24,xmeas_25,xmeas_26,xmeas_27,xmeas_28,xmeas_29,xmeas_30,xmeas_31,xmeas_32,xmeas_33,xmeas_34,xmeas_35,xmeas_36,xmeas_37,xmeas_38,xmeas_39,xmeas_40,xmeas_41,xmv_1,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,120.41,0.33818,80.044,51.435,2632.9,25.029,50.528,3101.1,22.819,65.732,229.61,341.22,94.640,77.047,32.188,8.8933,26.383,6.8820,18.776,1.6567,32.958,13.823,23.978,1.2565,18.579,2.2633,4.8436,2.2986,0.017866,0.83570,0.098577,53.724,43.828,62.881,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,75.000,120.41,0.33620,80.078,50.154,2633.8,24.419,48.772,3102.0,23.333,65.716,230.54,341.30,94.595,77.434,32.188,8.8933,26.383,6.8820,18.776,1.6567,32.958,13.823,23.978,1.2565,18.579,2.2633,4.8436,2.2986,0.017866,0.83570,0.098577,53.724,43.828,63.132,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,74.771,120.42,0.33563,80.220,50.302,2635.5,25.244,50.071,3103.5,21.924,65.732,230.08,341.38,94.605,77.466,31.767,8.7694,26.095,6.8259,18.961,1.6292,32.985,13.742,23.897,1.3001,18.765,2.2602,4.8543,2.3900,0.017866,0.83570,0.098577,53.724,43.828,63.117,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,75.224,120.39,0.33553,80.305,49.990,2635.6,23.268,50.435,3102.8,22.948,65.781,227.91,341.71,94.473,77.443,31.767,8.7694,26.095,6.8259,18.961,1.6292,32.985,13.742,23.897,1.3001,18.765,2.2602,4.8543,2.3900,0.017866,0.83570,0.098577,53.724,43.828,63.100,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,75.388,120.39,0.32632,80.064,51.310,2632.4,26.099,50.480,3103.5,22.808,65.788,231.37,341.11,94.678,76.947,32.322,8.5821,26.769,6.8688,18.782,1.6396,33.071,13.834,24.228,1.0938,18.666,2.2193,4.8304,2.2416,0.017866,0.83570,0.098577,53.724,43.828,63.313,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270395,20.0,24.0,496,0.28437,3645.3,4476.8,9.3158,27.041,42.017,2691.9,74.217,120.41,0.33693,79.951,50.860,2610.8,24.668,48.663,3093.0,22.379,65.982,235.45,336.23,94.645,76.700,32.044,8.7699,26.246,6.8068,19.346,1.6191,32.682,13.709,23.334,1.2064,18.913,2.2850,4.7707,2.2680,0.018453,0.85368,0.088404,53.842,44.153,63.581,53.831,27.820,58.953,19.810,40.356,40.631,43.441,49.613,42.208,19.711
270396,20.0,24.0,497,0.27925,3687.0,4471.1,9.2843,26.594,41.935,2692.1,74.513,120.39,0.32284,79.890,50.685,2610.6,27.534,49.249,3090.2,23.482,65.926,238.03,336.88,94.655,76.948,32.151,8.9606,26.211,6.8488,18.977,1.7243,32.865,13.787,23.628,1.0893,19.075,2.2680,4.8855,2.2031,0.018453,0.85368,0.088404,53.842,44.153,63.320,52.904,27.161,61.933,19.796,38.746,40.117,44.795,49.616,40.490,17.089
270397,20.0,24.0,498,0.27898,3671.8,4453.8,9.2279,26.427,42.475,2694.8,74.098,120.42,0.32033,80.184,49.269,2612.9,25.033,48.594,3092.7,23.109,65.937,238.28,337.30,94.665,77.357,32.151,8.9606,26.211,6.8488,18.977,1.7243,32.865,13.787,23.628,1.0893,19.075,2.2680,4.8855,2.2031,0.018453,0.85368,0.088404,53.842,44.153,63.581,53.112,27.188,62.026,19.764,39.052,35.948,43.279,49.824,41.805,17.934
270398,20.0,24.0,499,0.26937,3691.9,4495.6,9.3308,26.934,42.480,2694.8,75.163,120.39,0.33166,80.240,50.435,2612.6,25.598,48.453,3093.8,22.824,66.016,241.27,336.74,94.604,77.187,32.118,8.8653,26.227,6.8508,19.090,1.7002,32.6

In [17]:
x_train_df = Sampled_train.drop(['faultNumber','simulationRun','sample'],axis=1)
x_test_df = Sampled_test.drop(['faultNumber','simulationRun','sample'],axis =1)
x_cv_df = Sampled_cv.drop(['faultNumber','simulationRun','sample'],axis=1)

In [18]:
top10all = ['xmv_10', 'xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmv_5', 'xmeas_17', 'xmeas_18', 'xmeas_9']
len(top10all)

10

# LSTM

In [48]:
def x_generator(masked, feature_name):
    # generate the needed x matrix for training, while keeping x_df intact
    
    dimension = dict()
    
    # row dimension
    dimension['train_row'] = len(x_train_df)
    dimension['test_row'] = len(x_test_df)
    dimension['cv_row'] = len(x_cv_df)
    
    # decide whether to mask a feature or not
    if masked == True:
        x_train_masked_df = x_train_df.drop(feature_name, axis = 1)
        x_test_masked_df = x_test_df.drop(feature_name, axis = 1)
        x_cv_masked_df = x_cv_df.drop(feature_name, axis = 1)
    else:
        x_train_masked_df = x_train_df.copy()
        x_test_masked_df = x_test_df.copy()
        x_cv_masked_df = x_cv_df.copy()
    
    # column dimension
    dimension['train_col'] = x_train_masked_df.shape[1]
    dimension['test_col'] = x_test_masked_df.shape[1]
    dimension['cv_col'] = x_cv_masked_df.shape[1]
    
    standard_scalar = StandardScaler()
    x_train_masked_df = standard_scalar.fit_transform(x_train_masked_df)
    x_test_masked_df = standard_scalar.transform(x_test_masked_df)
    x_cv_masked_df = standard_scalar.transform(x_cv_masked_df)
    
    x_train = np.resize(x_train_masked_df, (dimension['train_row'], dimension['train_col'], 1))
    x_test = np.resize(x_test_masked_df, (dimension['test_row'], dimension['test_col'], 1))
    x_cv = np.resize(x_cv_masked_df, (dimension['cv_row'], dimension['cv_col'], 1))
    
    return dimension, x_train, x_test, x_cv

In [49]:
def train_model(x_train, y_train, x_cv, y_cv, train_col, feature_name):
    model = Sequential()
    model.add(LSTM(256,input_shape= (train_col, 1),return_sequences= True))
    model.add(LSTM(128,return_sequences= False))
    model.add(Dense(300))
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Dense(21,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print(model.summary())

    # training
    model.fit(x_train, y_train, epochs = 35, verbose=0,batch_size=256,validation_data = (x_cv, y_cv))
    
    # saving the model
    model.save('models/'+ feature_name)
    
    # saving the history
    model_paras = model.history
    with open('models/' + feature_name + '/history.pickle', 'wb') as handle:
        pickle.dump(model_paras.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

    !cp -r models/ /content/drive/MyDrive/

    return model

In [50]:
def visulize_model(history, feature_name):
    x = list(range(1,len(history['loss']) + 1))
    plt.figure(figsize=(5,5))
    plt.plot(x, history['val_accuracy'],color = 'r',label = 'Validation Accuracy')
    plt.plot(x, history['accuracy'],color = 'b',label = 'Training Accuracy')
    plt.grid()
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.savefig('models/' + feature_name + '/performance.png')
    plt.show()

In [51]:
# no longer needed!

def setupLSTM(toDrop, x_train, x_cv):
  base_train = x_train.drop(toDrop, axis=1)
  base_cv = x_cv.drop(toDrop, axis=1)

  standard_scalar = StandardScaler()
  x_train = standard_scalar.fit_transform(base_train)
  x_cv = standard_scalar.transform(base_cv)

  train_row = len(x_train)
  cv_row = len(x_cv)

  train_col = x_train.shape[1]
  cv_col = x_cv.shape[1]

  x_train = np.resize(x_train,(train_row, train_col, 1))
  x_cv = np.resize(x_cv,(cv_row, cv_col, 1))

  return x_train, x_cv

In [52]:
# no longer needed!

def createLstmModel(train_col):

  model = Sequential()
  model.add(LSTM(256,input_shape= (train_col,1),return_sequences= True))
  model.add(LSTM(128,return_sequences= False))
  model.add(Dense(300))
  model.add(Dropout(0.5))
  model.add(Dense(128))
  model.add(Dense(21,activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [55]:
dimension, x_train, x_test, x_cv = x_generator(masked = True, feature_name=top10all)
feature_name = 'base'
complete_model = train_model(x_train, y_train, x_cv, y_cv, dimension['train_col'], feature_name)

copy_features = top10all.copy()
score = max(complete_model.history.history['val_accuracy'])

print("Original Accuracy: %f\n" % score)
iterations = len(top10all)

for iter in range(iterations):
  print("Iteration: %d" % iter)

  best_feature = ""
  best_accuracy = 0
  copy_features = top10all.copy()

  for feature in top10all:
    copy_features.remove(feature)

    dimension, x_train, x_test, x_cv = x_generator(masked = True, feature_name=copy_features)
    feature_name = str(iter)+"_"+feature
    model = train_model(x_train, y_train, x_cv, y_cv, dimension['train_col'], feature_name)

    score = max(model.history.history['val_accuracy'])
    print("Feature %s Accuracy %f" % (feature, score))

    copy_features = top10all.copy()

    if score > best_accuracy:
      best_accuracy = score
      best_feature = feature
  
  print("Best Feature: %s" % best_feature)
  print("Accuracy is: %f\n" % best_accuracy)

  top10all.remove(best_feature)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 42, 256)           264192    
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_21 (Dense)             (None, 300)               38700     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_23 (Dense)             (None, 21)                2709      
Total params: 541,249
Trainable params: 541,249
Non-trainable params: 0
________________________________________________

INFO:tensorflow:Assets written to: models/base/assets


INFO:tensorflow:Assets written to: models/base/assets


Original Accuracy: 0.584536

Iteration: 0
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 43, 256)           264192    
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_24 (Dense)             (None, 300)               38700     
_________________________________________________________________
dropout_8 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_26 (Dense)             (None, 21)                2709      
Total params: 541,249
Trainable params: 541,249
Non-trainable params: 0
______

INFO:tensorflow:Assets written to: models/0_xmv_10/assets


...

INFO:tensorflow:Assets written to: models/8_xmeas_17/assets


Feature xmeas_17 Accuracy 0.880148
Best Feature: xmv_11
Accuracy is: 0.893515

Iteration: 9
Model: "sequential_62"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_124 (LSTM)              (None, 52, 256)           264192    
_________________________________________________________________
lstm_125 (LSTM)              (None, 128)               197120    
_________________________________________________________________
dense_186 (Dense)            (None, 300)               38700     
_________________________________________________________________
dropout_62 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_187 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_188 (Dense)            (None, 21)                2709      
Total params: 541,249
Train

INFO:tensorflow:Assets written to: models/9_xmeas_17/assets


INFO:tensorflow:Assets written to: models/9_xmeas_17/assets


Feature xmeas_17 Accuracy 0.917573
Best Feature: xmeas_17
Accuracy is: 0.917573



In [ ]:
train, cv = setupLSTM(top10all, x_train_df, x_cv_df)
model = createLstmModel(len(train[0]))
epochsNum = 110

print(model.summary())
model.fit(train, y_train, epochs=epochsNum,verbose=1,batch_size=256,validation_data = (cv, y_cv))
score = model.history.history['val_accuracy'][-1]
model.save('models2/base_42')
# !cp -r models2/ /content/drive/MyDrive/ # Only add if colab

copy_features = top10all.copy()

# print("Original Accuracy: %f\n" % score)
iterations = 10 #len(top10all)

# for iter in range(3,iterations):
#   print("Iteration: %d" % iter)

#   best_feature = ""
#   best_accuracy = 0
#   copy_features = top10all.copy()

#   for feature in top10all:
#     copy_features.remove(feature)
#     train, cv = setupLSTM(copy_features, x_train_df, x_cv_df)
#     model = createLstmModel(len(train[0]))
#     print(model.summary())

#     model.fit(train, y_train, epochs=epochsNum, verbose=0, batch_size=256, validation_data = (cv, y_cv))
#     score = model.history.history['val_accuracy'][-1]
#     print("Feature %s Accuracy %f" % (feature, score))
#     file_name = 'models2/' + str(42 + iter) + '_' + feature
#     model.save(file_name)
#     !cp -r models2/ /content/drive/MyDrive/

#     copy_features = top10all.copy()

#     if score > best_accuracy:
#       best_accuracy = score
#       best_feature = feature
  
#   print("Best Feature: %s" % best_feature)
#   print("Accuracy is: %f\n" % best_accuracy)

#   top10all.remove(best_feature)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 42, 256)           264192    
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_9 (Dense)              (None, 300)               38700     
_________________________________________________________________
dropout_3 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_11 (Dense)             (None, 21)                2709      
Total params: 541,249
Trainable params: 541,249
Non-trainable params: 0
________________________________________________

INFO:tensorflow:Assets written to: models2/base_42/assets


INFO:tensorflow:Assets written to: models2/base_42/assets


In [ ]:
from google.colab import files
# for var in top10all:
#   files.download('models/42_%s' % var)

# files.download('models/base_42')
# files.download('models/42_xmeas_22')
# files.download('models/42_xmeas_21')
!zip -r /content/file.zip /content/models
files.download('/content/file.zip')

updating: content/models/ (stored 0%)
updating: content/models/base_42/ (stored 0%)
updating: content/models/base_42/saved_model.pb (deflated 89%)
updating: content/models/base_42/variables/ (stored 0%)
updating: content/models/base_42/variables/variables.index (deflated 66%)
updating: content/models/base_42/variables/variables.data-00000-of-00001 (deflated 7%)
updating: content/models/base_42/assets/ (stored 0%)
updating: content/models/42_xmeas_35/ (stored 0%)
updating: content/models/42_xmeas_35/saved_model.pb (deflated 89%)
updating: content/models/42_xmeas_35/variables/ (stored 0%)
updating: content/models/42_xmeas_35/variables/variables.index (deflated 66%)
updating: content/models/42_xmeas_35/variables/variables.data-00000-of-00001 (deflated 7%)
updating: content/models/42_xmeas_35/assets/ (stored 0%)
updating: content/models/42_xmeas_17/ (stored 0%)
updating: content/models/42_xmeas_17/saved_model.pb (deflated 90%)
updating: content/models/42_xmeas_17/variables/ (stored 0%)
upd

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Testing
import keras
model = keras.models.load_model('/content/drive/MyDrive/base_42')
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 42, 256)           264192    
_________________________________________________________________
lstm_13 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_18 (Dense)             (None, 300)               38700     
_________________________________________________________________
dropout_6 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_20 (Dense)             (None, 21)                2709      
Total params: 541,249
Trainable params: 541,249
Non-trainable params: 0
________________________________________________